# 🔍 Inference with Saved State-LSTM + Random Forest Classification

This notebook loads previously saved state-based LSTM models, detects anomalies, and classifies the detected anomalies using a pre-trained Random Forest model.

## 📦 Step 1: Import Libraries

In [10]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import joblib
from tensorflow.keras.models import load_model


## 📥 Step 2: Load and Prepare Reshaped Data

In [11]:

df = pd.read_csv("../test_csv/motor_monitor_0_reshaped.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()


,timestamp,power,rpm,temperature,vibration,is_anomaly,state,label
0,2025-04-07 00:00:00,5.022329,2.653643,22.0,7.523581,False,off,Normal
1,2025-04-07 00:01:00,5.077981,2.226890,22.0,7.324660,False,off,Normal
2,2025-04-07 00:02:00,4.833379,2.293508,22.0,7.432946,False,off,Power_outage
3,2025-04-07 00:03:00,6.071584,2.802438,22.0,7.827574,False,off,Normal
4,2025-04-07 00:04:00,4.644964,2.028206,22.0,7.241896,False,off,Power_outage


## ⚙️ Step 3: Define Sliding Window Anomaly Detector

In [12]:

def create_sequences(data, seq_length=30):
    return np.array([data.iloc[i:i+seq_length].values for i in range(len(data)-seq_length)])

def detect_state_anomalies(df_state, model_path, feature_cols):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df_state[feature_cols])
    sequences = create_sequences(pd.DataFrame(scaled))
    
    model = load_model(model_path, compile=False)
    reconstructions = model.predict(sequences)
    errors = np.mean((reconstructions - sequences[:, -reconstructions.shape[1]:, :]) ** 2, axis=(1, 2))
    
    threshold = np.percentile(errors, 95)
    print(f"Threshold for {model_path}: {threshold:.6f}")
    
    flags = np.array([False] * len(df_state))
    flags[30:len(errors)+30] = errors > threshold
    df_state['reconstruction_error'] = [0.0]*30 + errors.tolist()
    df_state['is_anomaly'] = flags
    return df_state


## 🧠 Step 4: Apply Detection by State

In [13]:

df_all = pd.DataFrame()
feature_cols = ['power', 'rpm', 'temperature', 'vibration']
model_paths = {
    'run': "../test_cases/saved_models/lstm_run.h5",
    'idle': "../test_cases/saved_models/lstm_idle.h5",
    'off': "../test_cases/saved_models/lstm_off.h5"
}

for state in df['state'].unique():
    df_state = df[df['state'] == state].copy()
    model_path = model_paths.get(state)
    if model_path:
        df_state = detect_state_anomalies(df_state, model_path, feature_cols)
        df_all = pd.concat([df_all, df_state])
df = df_all.sort_values('timestamp').reset_index(drop=True)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step
Threshold for ../test_cases/saved_models/lstm_off.h5: 0.047103
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step
Threshold for ../test_cases/saved_models/lstm_idle.h5: 0.048351
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Threshold for ../test_cases/saved_models/lstm_run.h5: 298.217598


## 🤖 Step 5: Classify Detected Anomalies with Random Forest

In [14]:

# One-hot encode state to match training
df_encoded = pd.get_dummies(df, columns=["state"])
for col in ["state_off", "state_run", "state_idle"]:
    if col not in df_encoded.columns:
        df_encoded[col] = 0

# Load RF model
rf_model = joblib.load("../test_cases/saved_models/random_forest.pkl")

# Filter anomaly rows
anomaly_df = df_encoded[df_encoded["is_anomaly"] == True].copy()
features = ['power', 'rpm', 'temperature', 'vibration', 'state_off', 'state_run', 'state_idle']

# Evaluate
if 'label' in anomaly_df.columns:
    X_anomaly = anomaly_df[features]
    y_true = anomaly_df['label']
    y_pred = rf_model.predict(X_anomaly)
    
    print("🔍 Classification Report (for detected anomalies):")
    print(classification_report(y_true, y_pred))
else:
    print("⚠️ 'label' column not found. Cannot compute classification report.")


🔍 Classification Report (for detected anomalies):
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00        67
Power_outage       1.00      1.00      1.00         1

    accuracy                           1.00        68
   macro avg       1.00      1.00      1.00        68
weighted avg       1.00      1.00      1.00        68



c:\Users\Thomas\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
